<a href="https://colab.research.google.com/github/Thanh-Hoo/Pothole-Detection/blob/master/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Jun 27 13:01:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
'Colab Notebooks'  'Đồ án.zip'	 Poster.pdf   yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13775, done.
remote: Total 13775 (delta 0), reused 0 (delta 0), pack-reused 13775
Receiving objects: 100% (13775/13775), 12.35 MiB | 22.59 MiB/s, done.
Resolving deltas: 100% (9404/9404), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-06-27 13:04:37--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.90MB/s    in 3m 7s   

2020-06-27 13:07:45 (850 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
   creating: data/obj/images/
  inflating: data/obj/images/img-1.jpg  
  inflating: data/obj/images/img-1.txt  
  inflating: data/obj/images/img-10.jpg  
  inflating: data/obj/images/img-10.txt  
  inflating: data/obj/images/img-100.jpg  
  inflating: data/obj/images/img-100.txt  
  inflating: data/obj/images/img-101.jpg  
  inflating: data/obj/images/img-101.txt  
  inflating: data/obj/images/img-102.jpg  
  inflating: data/obj/images/img-102.txt  
  inflating: data/obj/images/img-103.jpg  
  inflating: data/obj/images/img-103.txt  
  inflating: data/obj/images/img-104.jpg  
  inflating: data/obj/images/img-104.txt  
  inflating: data/obj/images/img-105.jpg  
  inflating: data/obj/images/img-105.txt  
  inflating: data/obj/images/img-106.jpg  
  inflating: data/obj/images/img-106.txt  
  inflating: data/obj/images/img-107.jpg  
  inflating: data/obj/images/img-107.txt  
  inflating: data/obj/images/img-108.jpg  
  inflating: data/obj/images/img-108

In [ ]:
cd ..

/content


In [ ]:
cd darknet/

/content/darknet


In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.6868055555555556 0.3819444444444444 0.6125 0.30277777777777776
['0', '0.6868055555555556', '0.3819444444444444', '0.6125', '0.30277777777777776']
0 data/obj/images/img-205.txt
0 0.6868055555555556 0.3819444444444444 0.6125 0.30277777777777776
0 0.6044444444444445 0.7333333333333333 0.4622222222222222 0.5333333333333333
['0', '0.6044444444444445', '0.7333333333333333', '0.4622222222222222', '0.5333333333333333']
1 data/obj/images/img-541.txt
0 0.6044444444444445 0.7333333333333333 0.4622222222222222 0.5333333333333333
0 0.7532894736842105 0.45666666666666667 0.4276315789473684 0.14666666666666667
['0', '0.7532894736842105', '0.45666666666666667', '0.4276315789473684', '0.14666666666666667']
2 data/obj/images/img-303.txt
0 0.7532894736842105 0.45666666666666667 0.4276315789473684 0.14666666666666667
0 0.5166666666666667 0.4895833333333333 0.3 0.1763888888888889
['0', '0.5166666666666667', '0.4895833333333333', '0.3', '0.1763888888888889']
3 data/obj/images/img-9.txt
0 0.5166666666666

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/img-40.jpg', 'data/obj/images/img-544.jpg', 'data/obj/images/img-424.jpg', 'data/obj/images/img-165.jpg', 'data/obj/images/img-636.jpg', 'data/obj/images/img-361.jpg', 'data/obj/images/img-437.jpg', 'data/obj/images/img-577.jpg', 'data/obj/images/img-337.jpg', 'data/obj/images/img-23.jpg', 'data/obj/images/img-370.jpg', 'data/obj/images/img-172.jpg', 'data/obj/images/img-97.jpg', 'data/obj/images/img-349.jpg', 'data/obj/images/img-298.jpg', 'data/obj/images/img-488.jpg', 'data/obj/images/img-606.jpg', 'data/obj/images/img-31.jpg', 'data/obj/images/img-122.jpg', 'data/obj/images/img-385.jpg', 'data/obj/images/img-557.jpg', 'data/obj/images/img-504.jpg', 'data/obj/images/img-365.jpg', 'data/obj/images/img-94.jpg', 'data/obj/images/img-32.jpg', 'data/obj/images/img-355.jpg', 'data/obj/images/img-586.jpg', 'data/obj/images/img-45.jpg', 'data/obj/images/img-203.jpg', 'data/obj/images/img-401.jpg', 'data/obj/images/img-310.jpg', 'data/obj/images/img-626.jpg', 'data/obj/imag

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 247171, rewritten_bbox = 0.000000 % 

 3926: 0.060578, 0.091231 avg loss, 0.001000 rate, 5.333633 seconds, 251264 images, 0.158316 hours left
Loaded: 0.000045 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.893382, GIOU: 0.891416), Class: 0.999645, Obj: 0.981673, No Obj: 0.001652, .5R: 1.000000, .75R: 1.000000, count: 4, class_loss = 0.000888, iou_loss = 0.062818, total_loss = 0.063705 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
 tota